# SQL

**Описание задания**:

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Поэтому, компания купила крупный сервис для чтения книг по подписке. 

Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи**:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


**Описание данных**:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

###  Подключение к базе, чтение файлов с данными

Импортируем библиотеки.

In [1]:
#pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, будем использовать Pandas
#pd.io.sql.read_sql(query, con = engine)

Исследуем таблицы — выведием первые строки.

In [4]:
# данные таблицы authors
query = '''
            SELECT * FROM authors
        '''
# подключения к базе данных для Sqlite
authors = pd.io.sql.read_sql(query, con = engine)

authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# данные таблицы books
query = '''
            SELECT * FROM books
        '''
# подключения к базе данных для Sqlite
books = pd.io.sql.read_sql(query, con = engine)

books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
# данные таблицы publishers
query = '''
            SELECT * FROM publishers
        '''
# подключения к базе данных для Sqlite
publishers = pd.io.sql.read_sql(query, con = engine)

publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# данные таблицы ratings
query = '''
            SELECT * FROM ratings
        '''
# подключения к базе данных для Sqlite
ratings = pd.io.sql.read_sql(query, con = engine)

ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
# данные таблицы reviews
query = '''
            SELECT * FROM reviews
        '''
# подключения к базе данных для Sqlite
reviews = pd.io.sql.read_sql(query, con = engine)

reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [9]:
# выведем общую информацию о датасетах
list=[authors, books, publishers, ratings, reviews]

for i in list:
    print(i.info())
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  --

Изучение датасетов завершено, можно приступать к решению задач.

#### Задача №1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [11]:
# запрос
query = '''
SELECT COUNT(DISTINCT(book_id))
FROM books
WHERE publication_date > '2000-01-01'
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,count
0,819


После 01.01.2000г вышло 819 книг.

#### Задача №2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [12]:
# запрос
query = '''
SELECT books.book_id as books,
       books.title,
       (SELECT COUNT(review_id) as cnt_reviews
       FROM reviews
       WHERE book_id = books.book_id),
       (SELECT AVG(rating) 
       FROM ratings
       WHERE book_id = books.book_id)      
FROM books,
     reviews
GROUP BY books
ORDER BY cnt_reviews DESC
LIMIT 5
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,books,title,cnt_reviews,avg
0,948,Twilight (Twilight #1),7,3.662500
1,497,Outlander (Outlander #1),6,4.125000
2,207,Eat Pray Love,6,3.395833
3,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


Самое большое количество обзоров у книги "Сумерки".

#### Задача №3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [13]:
# запрос
query = '''
SELECT publishers.publisher,
       COUNT(books.book_id) AS number
FROM books
     LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50
GROUP BY publishers.publisher
ORDER BY number DESC
LIMIT 1
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,publisher,number
0,Penguin Books,42


Наибольшее количество книг толще 50 страниц выпустило издание Penguin Books.

#### Задача №4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [14]:
# запрос
query = '''
SELECT authors.author,
       AVG(ratings.rating) AS avg_rating
FROM books
     LEFT JOIN authors ON books.author_id = authors.author_id
     LEFT JOIN ratings ON ratings.book_id = books.book_id
WHERE books.book_id IN 
                     (SELECT books.book_id
                      FROM books    
                        LEFT JOIN ratings ON ratings.book_id = books.book_id     
                     GROUP BY books.book_id
                     HAVING COUNT(ratings.rating_id) >= 50)
GROUP BY authors.author  
ORDER BY avg_rating DESC
LIMIT 1
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Наивысший средний рейтинг у J.K. Rowling и Mary GrandPré.

#### Задача №5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [15]:
# запрос
query = '''
SELECT FLOOR(AVG(rev.review_count)) AS average_review_count
FROM (SELECT COUNT(review_id) AS review_count
      FROM reviews
      WHERE username IN (
            SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50)
GROUP BY username 
    ) AS rev;
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,average_review_count
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.

#### Вывод

* Все таблицы исследованы, задачи решены. 
* Согласно заданию, сделано по одному SQL-запросу для решения каждого задания, результаты выведены.

Результы рещения задач:
- После 01.01.2000г вышло 819 книг.
- Самое большое количество обзоров у книги "Сумерки".
- Наибольшее количество книг толще 50 страниц выпустило издание Penguin Books.
- Наивысший средний рейтинг у J.K. Rowling и Mary GrandPré.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.